In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [2]:
def insert_sample(d,target_f,fs,sample):
    if(target_f in d):
        if(fs in d[target_f]):
            d[target_f][fs].append(sample)
        else:
            d[target_f][fs] = [sample]
    else:
        d[target_f] = {fs:[sample]}

def read_file():
    #read from file, expected frequency and sample array
    #run test, compare expected to processed result
    with open('dataset.txt') as f:
        sample_chunks =[]
        d=dict()
        target_f = 0
        fs = 0
        for line in f:
            line = line.replace(" ","")        
            if(line.lower().find("f=") != -1):
                target_f = (line[line.lower().find("f=")+2:line.find('\n')])
                #if(target_f.isnumeric()):
                #    target_f=np.float64(target_f)
                #else:
                #    target_f = 0
            if(line.lower().find("fs=") != -1):
                fs = np.uint16(line[line.lower().find("fs=")+3:line.find('\n')])

            data_start = line.find('[')
            if(data_start != -1):
                if(fs == 0):
                    print("warning, FS not declared")
                data_chunk = np.array(line[data_start+1:line.find(']')].split(',')).astype(np.int32)
                insert_sample(d,target_f,fs,data_chunk)
    return d    

In [3]:
def print_plot(fs,t,signal,target_f):
    f = 150
    if target_f.isnumeric():
        f = np.int32(target_f)    
    plt.subplot(2,2,1)
    plt.title("Source signal. sample length = %i " %(len(signal)))
    plt.plot(t, signal)
    plt.axhline(0,color='black')
    
    plt.subplot(2,2,2)    
    raw_signal_fft = np.fft.rfft(signal)
    raw_signal_fft_bins = np.fft.rfftfreq(signal.size, d=1/fs)
    max_peak = np.argmax(abs(raw_signal_fft))
    acc = ((fs/2)/(len(signal)/2))
    plt.title("FFT acc +/- %i Hz | dominant f %i Hz" %(acc,raw_signal_fft_bins[max_peak]))
    plt.axvline(raw_signal_fft_bins[max_peak],color="green")
    plt.plot(raw_signal_fft_bins,np.abs(raw_signal_fft))
    plt.xlim(0, f*5) #show range up to 4th harmonic    
    plt.tight_layout()
    plt.show()    

In [4]:
def print_result_plot(fs,t,result,target_f,t_idx):
    plt.subplot(2,2,1)
    plt.plot(t,result)
    plt.plot(t[t_idx],result[t_idx],'ro')
    plt.title("f[%i]=%.2f Hz | T=%.3f ms" %(t_idx,fs/t_idx,(t_idx*1000)/fs))    
    
    t_adj = get_interpolation(result,t_idx)
    plt.subplot(2,2,2)
    window = 5
    plt.plot(t[t_idx-window:t_idx+window+1],result[t_idx-window:t_idx+window+1])
    
    plt.plot(t_adj/fs,result[t_idx],"bo",label = 'adjusted center')
    plt.plot(t[t_idx+1],result[t_idx+1],"go",label = 'center +1')
    plt.plot(t[t_idx],result[t_idx],"ro",label='center')
    plt.plot(t[t_idx-1],result[t_idx-1],"go",label = 'center -1')
    plt.legend()
    plt.title("f[%.2f]=%.2f Hz | T=%.3f ms" %(t_adj,fs/t_adj,(t_adj*1000)/fs))
    plt.tight_layout()
    plt.show()

In [5]:
def acf(x):
    r = np.zeros(len(x))
    for k in range(len(x)):
        for j in range(len(x)-k):
            r[k] += (x[j]*x[j+k]) #acf
        #r[k] = r[k]/(len(x)+3*k)
        r[k] = r[k]/(len(x)+k)
        #r[k] = r[k]/(len(x))
    return r

In [6]:
def amdf(x):
    r = np.zeros(len(x))
    for k in range(0,len(x)):
        for j in range(len(x)-k):
            r[k] += abs(x[j]-x[j+k])
        #r[k] = r[k]/(len(x)-k)
        #r[k] = r[k]/len(x)
        #r[k] = r[k]+20*k
    return r

In [7]:
def get_idx(x,fs,corr_type):
    T_MIN = fs/365
    T_MAX = fs/65
    target_val = 0
    t_idx = -1
    
    if corr_type == "amdf":
        target_val = np.iinfo(np.int16).max
        
    for i in range(len(x)):
        if( i  > T_MIN and i < T_MAX):
            if corr_type == "acf":                
                if x[i] > target_val:
                    target_val = x[i]
                    t_idx = i 
            elif corr_type == "amdf":
                if x[i] < target_val:
                    target_val = x[i]
                    t_idx = i
    return t_idx
        

In [8]:
def get_interpolation(x,t_idx):
    p_adj = ((x[t_idx+1] - x[t_idx-1])/(2*(2*x[t_idx] - x[t_idx+1] - x[t_idx-1])))
    return t_idx+p_adj

In [9]:
def get_results(d,target_f,fs,corr_type):
    res_d={'res':[],'idx':[],'f':[],'error':[]}

    for i in range(len(d[target_f][fs])):
        signal = d[target_f][fs][i]
        res_d["res"].append(eval(corr_type)(signal))
        res_d["idx"].append(get_idx(res_d["res"][i],fs,corr_type))
        res_d["f"].append(fs/get_interpolation(res_d["res"][i],res_d["idx"][i]))
        if target_f.isnumeric():
            f = np.int32(target_f)
            res_d["error"].append(abs(f - res_d["f"][i]) > 2)
        
    return res_d

In [10]:
#widgets
def freq_select(target_f,d):
    def fs_select(fs):
        sample_size = len(d[target_f][fs])
        
        acf_res = get_results(d,target_f,fs,"acf")
        acf_err = sum(acf_res["error"])
        
        amdf_res = get_results(d,target_f,fs,"amdf")
        amdf_err = sum(amdf_res["error"])
        
        print("Num of samples in test",sample_size)
        print("ACF results \t\t AMDF results")        
        print("errors\t",acf_err,"\t\t",amdf_err)
        #print("error %3.2f\t\t%3.2f" %((acf_err/sample_size)*100, (amdf_err/sample_size)*100))
        print("error %\t {:3.2f} \t\t {:3.2f}".format((acf_err/sample_size)*100, (amdf_err/sample_size)*100))
        def slider(x):
            signal = d[target_f][fs][x-1]
            l = len(signal)/fs 
            t = np.arange(0,l,1.0/fs) #x axis time scale
            
            plt.rcParams['figure.figsize'] = [10, 4]
            print_plot(fs,t,signal,target_f)
            plt.suptitle("ACF")
            print_result_plot(fs,t,acf_res["res"][x-1],target_f,acf_res["idx"][x-1])
            plt.suptitle("AMDF")
            print_result_plot(fs,t,amdf_res["res"][x-1],target_f,amdf_res["idx"][x-1])
            
        slider_max = len(d[target_f][fs])    
        widgets.interact(slider, x= widgets.IntSlider(min=1,max =slider_max,step = 1));
    widgets.interact(fs_select,fs=list(d[target_f]))

d = read_file()
widgets.interact(freq_select,target_f = list(d.keys()),d=widgets.fixed(d))
    

interactive(children=(Dropdown(description='target_f', options=('82', '110', '146', '197', '246', '329', 'auto…

<function __main__.freq_select(target_f, d)>